In [206]:
from colmap_testing.colmap_helpers import read_write_model
import os
import cv2
import numpy as np
from constants import *
from matplotlib import pyplot as plt

#https://stackoverflow.com/questions/4813061/non-alphanumeric-list-order-from-os-listdir
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)


In [207]:
recording_id = "f5b09068-d395-493c-80e6-7eb0cbb6478a"
colmap_ws_path = os.path.join(recordings_folder, recording_id, "colmap_ws")
colmap_dataset_path = os.path.join(colmap_ws_path, "dataset")
colmap_output_path = os.path.join(colmap_ws_path, "colmap_out")

path_to_world_images = os.path.join(colmap_output_path,"dense", "images", "world_images")
path_to_depth_rgb_images = os.path.join(colmap_output_path,"dense", "images", "depth_images")

world_images = sorted_alphanumeric(os.listdir(path_to_world_images))
depth_images = sorted_alphanumeric(os.listdir(path_to_depth_rgb_images))

path_to_model = os.path.join(colmap_output_path, "sparse","0")
world_image_paths = [os.path.join(path_to_world_images, str(name) + ".png") for name in world_images]
depth_image_paths = [os.path.join(path_to_depth_rgb_images, str(name) + ".png") for name in depth_images]


In [208]:
world_images_array = np.array([np.array(cv2.imread(imagepath)) for imagepath in world_image_paths])
depth_images_array = np.array([np.array(cv2.imread(imagepath)) for imagepath in depth_image_paths])

colmap_model = read_write_model.read_model(path_to_model, ".bin")

#cameras:fx, fy, cx, cy, width, height
#images:camera_id, qvec, tvec, name 
#points:xyz, rgb, error, image_ids, point2D_idxs
cameras, images, points = read_write_model.read_model(path_to_model, ".bin")

# 3D point list with one line of data per point:
# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
points_keys = list(points.keys())

image_ids_visible_point = []
while 1 not in image_ids_visible_point or len(image_ids_visible_point) > 10:
    random_key = np.random.choice(points_keys)
    random_point = points[random_key]
    random_point_3d = random_point.xyz
    image_ids_visible_point = random_point.image_ids

print(image_ids_visible_point)

# Image list with two lines of data per image:
#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
#   POINTS2D[] as (X, Y, POINT3D_ID)

#first_image = images[1]

# Camera list with one line of data per camera:
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
#first_camera = cameras[1]



#The coordinates of the projection/camera center are given by -R^t * T, 
# where R^t is the inverse/transpose of the 3x3 rotation matrix composed from the quaternion and T is the translation vector. 
# The local camera coordinate system of an image is defined in a way that the X axis points to the right, 
# the Y axis to the bottom, and the Z axis to the front as seen from the image.

#calculate the projection/camera center

[ 5 40 11 10 45  1]


In [209]:
def get_frames(images, cameras, scale=1):
    frames = []
    keys = images.keys()
    
    for key in keys:
        img = images[key]
        # rotation
        R = read_write_model.qvec2rotmat(img.qvec)

        # translation
        t = img.tvec

        # invert
        t = -R.T @ t
        R = R.T

        # intrinsics
        cam = cameras[img.camera_id]
        if cam.model in ("SIMPLE_PINHOLE", "SIMPLE_RADIAL", "RADIAL"):
            fx = fy = cam.params[0]
            cx = cam.params[1]
            cy = cam.params[2]
        elif cam.model in ("PINHOLE", "OPENCV", "OPENCV_FISHEYE", "FULL_OPENCV"):
            fx = cam.params[0]
            fy = cam.params[1]
            cx = cam.params[2]
            cy = cam.params[3]
        else:
            raise Exception("Camera model not supported")

        # intrinsics
        K = np.identity(3)
        K[0, 0] = fx
        K[1, 1] = fy
        K[0, 2] = cx
        K[1, 2] = cy
        frames.append((key, R, t, K))
    return frames

In [210]:
visible_images = dict((image_id, images[image_id]) for image_id in random_point.image_ids)
frames = get_frames(visible_images, cameras)
if not os.path.exists(os.path.join(colmap_ws_path,"reprojections_test")):
    os.mkdir(os.path.join(colmap_ws_path,"reprojections_test"))
for frame in frames:
    image_id, R, t, K = frame
    image_name = visible_images[image_id].name
    path_to_image = os.path.join(colmap_output_path,"dense", "images", image_name)
    image_array = cv2.imread(path_to_image)
    #add nested to t
    t = np.array([t]).T
    P = K @ np.hstack((R, t)) #projection matrix
    point_on_image = P @ np.hstack((random_point_3d, 1))
    point_on_image = point_on_image / point_on_image[2]
    print(image_dimensions[1]-point_on_image[0], image_dimensions[0]-point_on_image[1])
    image_dimensions = image_array.shape
    plt.scatter(image_dimensions[1]-point_on_image[0],image_dimensions[0]-point_on_image[1], c="r")
    plt.imshow(image_array)
    plt.savefig(os.path.join(colmap_ws_path, "reprojections_test", str(image_id) + ".png"))
    #plt.show()
    plt.close()

237.59553150247064 -71.79501775739232
522.8217600735411 485.1430875223722
498.0541267526619 511.6820991308756
364.6360768686102 519.4699551535773
93.09794314875137 466.38281040288086
781.2025393759093 936.7790128339354
